In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
scala_version = '2.13'
spark_version = '3.5.5'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}','org.apache.kafka:kafka-clients:2.10.0']
spark = SparkSession.builder.master('local')\
.appName("kafka-example").config("spark.jars.packages", ",".join(packages))\
.config("spark.memory.offHeap.enabled","true")\
.config("spark.memory.offHeap.size","10g")\
.config("spark.driver.memory", "16g")\
.getOrCreate()

In [2]:
topic_name = 'StreetviewImages'
kafka_server = 'localhost:9092'

streamRawDf = spark.readStream.format("kafka")\
    .option("kafka.bootstrap.servers", kafka_server)\
    .option("subscribe", topic_name)\
    .load()

streamDF = streamRawDf.select(col('topic'),col('offset'),col('value').cast('string'))


In [3]:
stream_writer0 = (streamDF.writeStream.queryName("CameraStreaming0").trigger(processingTime="5 seconds").outputMode("append").format("memory"))

query0 = stream_writer0.start()   

In [4]:
from time import sleep
from IPython.display import display, clear_output
for x in range(0, 100):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        result0 = spark.sql(f"SELECT * FROM {query0.name}")

        display(result0.toPandas())

        sleep(3)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended...")


Showing live view refreshed every 5 seconds
Seconds passed: 30


,topic,offset,value
0,StreetviewImages,4,"{""image_id"": ""10.7679053_106.6677161.jpg"", ""la..."
1,StreetviewImages,5,"{""image_id"": ""10.7679184_106.6677675.jpg"", ""la..."
2,StreetviewImages,6,"{""image_id"": ""10.7679274_106.6678216.jpg"", ""la..."
3,StreetviewImages,7,"{""image_id"": ""10.7680118_106.6678825.jpg"", ""la..."


break
Live view ended...
